In [1]:
import pandas as pd 

es_df = pd.read_csv('ES_meta.csv')
es_df



,filename,fold,target,category
0,./data/wav_정상품_V1a/#01_SX2_Normal_ES_RH_1/#01_...,1,0,Normal
1,./data/wav_정상품_V1a/#01_SX2_Normal_ES_RH_2/#01_...,1,0,Normal
2,./data/wav_정상품_V1a/#01_SX2_Normal_ES_LH_1/#01_...,1,0,Normal
3,./data/wav_정상품_V1a/#01_SX2_Normal_ES_LH_2/#01_...,1,0,Normal
4,./data/wav_정상품_V1a/#02_SX2_Normal_ES_LH_1/#02_...,1,0,Normal
...,...,...,...,...
540,./data/wav_문제품DATAa/#44_SX2_C_ES_LH_1/#44_SX2_...,1,3,SG
541,./data/wav_문제품DATAa/#33_SX2_B_ES_RH_1/#33_SX2_...,1,3,SG
542,./data/wav_문제품DATAa/#33_SX2_B_ES_RH_2/#33_SX2_...,1,3,SG
543,./data/wav_문제품DATAa/#21_SX2_A_ES_RH_1/#21_SX2_...,1,3,SG


In [3]:
line_df = pd.read_csv('LINE_meta.csv')
line_df

,filename,fold,target,category
0,./data/wav_정상품_V1a/#01_SX2_Normal_Line_RH_2/#0...,1,0,Normal
1,./data/wav_정상품_V1a/#01_SX2_Normal_Line_RH_1/#0...,1,0,Normal
2,./data/wav_정상품_V1a/#01_SX2_Normal_Line_LH_2/#0...,1,0,Normal
3,./data/wav_정상품_V1a/#01_SX2_Normal_Line_LH_1/#0...,1,0,Normal
4,./data/wav_정상품_V1a/#02_SX2_Normal_Line_RH_2/#0...,1,0,Normal
...,...,...,...,...
545,./data/wav_문제품DATAa/#32_SX2_A_LINE_LH_1/#32_SX...,1,3,SG
546,./data/wav_문제품DATAa/#33_SX2_B_LINE_RH_2/#33_SX...,1,3,SG
547,./data/wav_문제품DATAa/#33_SX2_B_LINE_RH_1/#33_SX...,1,3,SG
548,./data/wav_문제품DATAa/#21_SX2_A_LINE_RH_2/#21_SX...,1,3,SG


In [4]:
import librosa
import numpy as np
import torch
# .wav 파일 불러오기


# 예제 사용

def get_tensor(file_path):
    audio, sr = librosa.load(file_path, sr=None)  # sr=None을 설정하여 원본 샘플링 레이트를 유지

    # NumPy 배열을 PyTorch 텐서로 변환
    tensor_data = torch.tensor(audio)

    # 결과 확인
    def perform_stft(waveform, n_fft=512, win_length=None, hop_length=None, window_fn=torch.hann_window):
        stft = torch.stft(waveform, n_fft=n_fft, win_length=win_length, hop_length=hop_length, window=window_fn(n_fft), return_complex=True)
        return stft

    stft_result = perform_stft(tensor_data)

    def apply_a_weighting_librosa(stft_result, sample_rate):
        # STFT 결과로부터 FFT 크기 추정
        n_fft = (stft_result.size(0) - 1) * 2
        
        # 주파수 bins 계산
        freqs = np.linspace(0, sample_rate / 2, stft_result.size(0))
        
        # A-weighting dB 값 계산
        a_weighting_db = librosa.A_weighting(freqs)
        
        # dB를 power scale로 변환
        a_weighting_scale = librosa.db_to_amplitude(a_weighting_db)
        
        # 텐서로 변환
        a_weighting_scale_tensor = torch.from_numpy(a_weighting_scale).to(stft_result.dtype)
        
        # A-weighted STFT 계산
        a_weighted_stft = torch.abs(stft_result).log10() + a_weighting_scale_tensor.unsqueeze(1)  # 주파수 차원에 적용
        #a_weighted_stft = torch.abs(stft_result) * a_weighting_scale_tensor.unsqueeze(1)  # 주파수 차원에 적용

        return a_weighted_stft

    sample_rate = 12800  # 샘플 레이트 예제 값
    # A-weighting 적용
    a_weighted_result = apply_a_weighting_librosa(stft_result, sample_rate)
    _3k_result = a_weighted_result[4:129,:]
    return torch.abs(_3k_result)


In [6]:
es_meta_normal_list = []
es_meta_abnormal_list = []

es_abnormal_name= []

for idx in es_df.index:
    row = es_df.loc[idx]
    filename = row['filename']
    if row['category'] == 'Normal':
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            es_meta_normal_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)
    else:
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            es_meta_abnormal_list.append(ten[:,:1000].unsqueeze(0))
            es_abnormal_name.append(filename)
        else:
            print(ten.shape)


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/librosa/core/convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


torch.Size([125, 349])
torch.Size([125, 596])
torch.Size([125, 587])
torch.Size([125, 367])


In [158]:
rps_df = pd.read_csv('1.0RPS_meta.csv')

rps_normal_list = []
rps_abnormal_list = []

for idx in rps_df.index:
    row = rps_df.loc[idx]
    filename= row['filename']
    if row['category'] == 'Normal':
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
           rps_normal_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)
    else:
        ten = get_tensor(filename)
        if ten.shape[1]>=1000:
            rps_abnormal_list.append(ten[:,:1000].unsqueeze(0))
        else:
            print(ten.shape)


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/librosa/core/convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__() # (125, 1000) 이라고 했을 때 
        # Encoder
        self.encoder = nn.Sequential( # Given groups=1, weight of size [16, 128, 3, 3], expected input[1, 32, 8, 95] to have 128 channels, but got 32 channels instead
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 16, 63, 500)
            nn.ReLU(True),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 32, 32, 250)
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 64, 16, 125)
            nn.ReLU(True),
            nn.Conv2d(64, 32, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 32, 8, 63)
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 16, 4, 32)
            nn.ReLU(True),
            nn.Conv2d(16, 1, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 1, 2, 16)
            #nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 16, kernel_size=3, stride=2, padding=1, output_padding=1),  # Output shape: (b, 16, 4, 32)  
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 32, kernel_size=3, stride=2, padding=(1, 1), output_padding=(1, 0)),  # Output shape: (b, 32, 8, 63) 
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 64, kernel_size=3, stride=2, padding=(1,1), output_padding=(1,0)),  # Output shape: (b, 64, 16, 125) 
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # Output shape: (b, 32, 32, 250)  
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 16, 63, 500)  
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 1, 125, 1517) 
            #nn.Tanh()
        )
# output padding must be smaller than stride
    def forward(self, x):
        mid = self.encoder(x)
        x = self.decoder(mid)
        return x
    
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


test_autoencoder = Autoencoder()

input_tensor = torch.randn(1, 1, 125, 1000)  # (batch_size, channels, height, width)

# Forward pass through the autoencoder
output_tensor = test_autoencoder(input_tensor)

# Print the shapes of input and output tensors
print("Input Tensor Shape:", input_tensor.shape)
print("Output Tensor Shape:", output_tensor.shape)



Input Tensor Shape: torch.Size([1, 1, 125, 1000])
Output Tensor Shape: torch.Size([1, 1, 125, 1000])


In [21]:
from torchsummary import summary
summary(test_autoencoder, (1, 125, 1000), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 63, 500]             160
              ReLU-2          [-1, 16, 63, 500]               0
            Conv2d-3          [-1, 32, 32, 250]           4,640
              ReLU-4          [-1, 32, 32, 250]               0
            Conv2d-5          [-1, 64, 16, 125]          18,496
              ReLU-6          [-1, 64, 16, 125]               0
            Conv2d-7            [-1, 32, 8, 63]          18,464
              ReLU-8            [-1, 32, 8, 63]               0
            Conv2d-9            [-1, 16, 4, 32]           4,624
             ReLU-10            [-1, 16, 4, 32]               0
           Conv2d-11             [-1, 1, 2, 16]             145
  ConvTranspose2d-12            [-1, 16, 4, 32]             160
             ReLU-13            [-1, 16, 4, 32]               0
  ConvTranspose2d-14            [-1, 32

In [233]:
def train(dataset_list):
    dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    customdataset = CustomDataset(dataset_list)
    train_loader = DataLoader(customdataset, batch_size=32, shuffle=True)


    # Initialize the model, loss function, and optimizer
    autoencoder = Autoencoder().to(dev)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

    # Train the autoencoder
    num_epochs = 100
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader): # i-th batch
            inputs = data
            inputs = inputs.to(dev)

            optimizer.zero_grad()

            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'epoch : {epoch}, loss: {running_loss}')


    print('Finished Training')

    encoded_list = []
    with torch.no_grad():
        for i in dataset_list:
            encoded_list.append(autoencoder.encoder(i.to(dev)).flatten())

    return encoded_list
           


In [234]:
encoded_es_normal_list = train(es_meta_normal_list)

epoch : 0, loss: 12.398808717727661
epoch : 1, loss: 11.313665509223938
epoch : 2, loss: 7.6079405546188354
epoch : 3, loss: 3.606076329946518
epoch : 4, loss: 2.3205993622541428
epoch : 5, loss: 1.6610265374183655
epoch : 6, loss: 1.350698471069336
epoch : 7, loss: 1.1978339776396751
epoch : 8, loss: 1.10973010212183
epoch : 9, loss: 1.0824408382177353
epoch : 10, loss: 1.061182513833046
epoch : 11, loss: 1.048851229250431


KeyboardInterrupt: 

In [235]:
encoded_es_normal_list = train(es_meta_normal_list)
encoded_es_abnormal_list = train(es_meta_abnormal_list)
# encoded_line_normal_list = train(line_meta_normal_list)
# encoded_line_abnormal_list = train(line_meta_abnormal_list)
encoded_rps_normal_list = train(rps_normal_list)
encoded_rps_abnormal_list = train(rps_abnormal_list)

epoch : 0, loss: 11.079938650131226
epoch : 1, loss: 10.131383061408997
epoch : 2, loss: 7.196853876113892
epoch : 3, loss: 4.115974992513657
epoch : 4, loss: 2.647664353251457
epoch : 5, loss: 1.7720158100128174
epoch : 6, loss: 1.3735427930951118
epoch : 7, loss: 1.1650108620524406
epoch : 8, loss: 1.078776240348816
epoch : 9, loss: 1.0462524518370628
epoch : 10, loss: 1.031832866370678
epoch : 11, loss: 1.0220938175916672
epoch : 12, loss: 1.0143959671258926
epoch : 13, loss: 1.0079897344112396
epoch : 14, loss: 1.0031520798802376
epoch : 15, loss: 0.9975587651133537
epoch : 16, loss: 0.9931326434016228
epoch : 17, loss: 0.9905939921736717
epoch : 18, loss: 0.9859994798898697
epoch : 19, loss: 0.9844277128577232
epoch : 20, loss: 0.9813613072037697
epoch : 21, loss: 0.9787669256329536
epoch : 22, loss: 0.9770009964704514
epoch : 23, loss: 0.9734100177884102
epoch : 24, loss: 0.9718736410140991
epoch : 25, loss: 0.9693892598152161
epoch : 26, loss: 0.9664949178695679
epoch : 27, loss

In [160]:
print(len(encoded_es_normal_list))
print(len(encoded_es_abnormal_list))

298
243


In [242]:
idx = 13
print(encoded_es_normal_list[idx])
print(encoded_es_abnormal_list[idx])

tensor([-25.0180, -20.0861, -20.0657, -20.3882, -19.5037, -18.2113, -18.6844,
        -17.7336, -17.3145, -16.8534, -16.1940, -14.3780, -14.2186, -13.2293,
        -12.3793, -11.3426, -34.0599, -33.6596, -29.7357, -26.8967, -25.9233,
        -22.5421, -20.4267, -18.6510, -17.0051, -15.6653, -14.7202, -13.6317,
        -12.9097, -11.6508, -10.8183, -10.1089], device='cuda:0')
tensor([-45.1905, -44.3485, -44.1869, -37.8495, -36.1261, -33.4207, -31.1672,
        -33.2614, -30.1946, -30.0322, -27.9996, -27.2515, -23.4164, -16.5871,
        -15.1389, -16.9202, -69.3491, -64.7952, -58.7570, -46.1870, -34.1557,
        -25.0276, -18.7406, -14.1415, -12.2227, -10.3272, -12.7524,  -7.1964,
          1.7215,  12.8773,  16.1953,  12.0615], device='cuda:0')


In [189]:
print(len(encoded_rps_normal_list))
print(len(encoded_rps_abnormal_list))

145
102


In [228]:
idx = 32
print(encoded_rps_normal_list[idx])
print(encoded_rps_abnormal_list[idx])

tensor([0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157,
        0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157,
        0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157, 0.7157,
        0.7157, 0.7157, 0.7157, 0.7157, 0.7157], device='cuda:0')
tensor([22.0828, 29.9187, 28.6448, 29.7196, 29.2427, 29.5657, 29.6560, 29.6708,
        29.3107, 29.6903, 28.8966, 29.5175, 28.5672, 29.0806, 29.6665, 23.6228,
        15.2068, 23.2690, 23.2540, 23.1654, 22.2447, 23.2502, 23.8726, 22.6562,
        22.7802, 22.3268, 23.6160, 22.3855, 23.0119, 19.4682, 18.8828, 23.2763],
       device='cuda:0')


In [259]:
from sklearn.cluster import KMeans

es_concated = encoded_es_normal_list + encoded_es_abnormal_list
es_concated = torch.stack(es_concated).cpu().numpy()

# Run k-means clustering
kmeans = KMeans(n_clusters=2, random_state=0)
es_cluster_labels = kmeans.fit_predict(es_concated)




/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [260]:
es_cluster_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [11]:
index = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1]

for i in index:
    if i == 0:
        print(es_abnormal_name[i])

./data/wav_문제품DATAa/#20_SX2_C_ES_RH_2/#20_SX2_C_ES_RH_2_3_Mic_30cm_S.wav
./data/wav_문제품DATAa/#20_SX2_C_ES_RH_2/#20_SX2_C_ES_RH_2_3_Mic_30cm_S.wav
./data/wav_문제품DATAa/#20_SX2_C_ES_RH_2/#20_SX2_C_ES_RH_2_3_Mic_30cm_S.wav
./data/wav_문제품DATAa/#20_SX2_C_ES_RH_2/#20_SX2_C_ES_RH_2_3_Mic_30cm_S.wav
./data/wav_문제품DATAa/#20_SX2_C_ES_RH_2/#20_SX2_C_ES_RH_2_3_Mic_30cm_S.wav
./data/wav_문제품DATAa/#20_SX2_C_ES_RH_2/#20_SX2_C_ES_RH_2_3_Mic_30cm_S.wav
./data/wav_문제품DATAa/#20_SX2_C_ES_RH_2/#20_SX2_C_ES_RH_2_3_Mic_30cm_S.wav
./data/wav_문제품DATAa/#20_SX2_C_ES_RH_2/#20_SX2_C_ES_RH_2_3_Mic_30cm_S.wav


In [264]:
es_normal_cnt={0:0, 1:0} # total should be: 298
es_abnormal_cnt={0:0, 1:0} # 243
for i, cl_label in enumerate(es_cluster_labels):
    if i < len(encoded_es_normal_list):
        es_normal_cnt[cl_label]+= 1
    else:
        es_abnormal_cnt[cl_label] += 1
    


In [265]:
print(es_normal_cnt)
print(es_abnormal_cnt)


{0: 298, 1: 0}
{0: 8, 1: 235}


In [269]:
rps_concated = encoded_rps_normal_list + encoded_rps_abnormal_list
rps_concated = torch.stack(rps_concated).cpu().numpy()

# Run k-means clustering
rps_cluster_labels = kmeans.fit_predict(rps_concated)

rps_normal_cnt={0:0, 1:0} # total should be: 145
rps_abnormal_cnt={0:0, 1:0} # 102

for i, cl_label in enumerate(rps_cluster_labels):
    if i < len(encoded_rps_normal_list):
        rps_normal_cnt[cl_label]+= 1
    else:
        rps_abnormal_cnt[cl_label] += 1
    
print(rps_normal_cnt)
print(rps_abnormal_cnt)


{0: 145, 1: 0}
{0: 0, 1: 102}


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [287]:
line_df = pd.read_csv('LINE_meta.csv')

line_meta_normal_list = []
line_meta_abnormal_list = []

for idx in line_df.index:
    row = line_df.loc[idx]
    filename= row['filename']
    if row['category'] == 'Normal':
        ten = get_tensor(filename)
        if ten.shape[1]>=500:
            line_meta_normal_list.append(ten[:,:500].unsqueeze(0))
        else:
            print(ten.shape)
    else:
        ten = get_tensor(filename)
        if ten.shape[1]>=500:
            line_meta_abnormal_list.append(ten[:,:500].unsqueeze(0))
        else:
            print(ten.shape)


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/librosa/core/convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)


torch.Size([125, 367])
torch.Size([125, 459])
torch.Size([125, 437])
torch.Size([125, 404])
torch.Size([125, 403])
torch.Size([125, 461])
torch.Size([125, 447])
torch.Size([125, 397])
torch.Size([125, 403])
torch.Size([125, 362])
torch.Size([125, 362])
torch.Size([125, 363])
torch.Size([125, 461])
torch.Size([125, 493])
torch.Size([125, 372])
torch.Size([125, 456])
torch.Size([125, 481])
torch.Size([125, 455])
torch.Size([125, 376])
torch.Size([125, 372])
torch.Size([125, 442])
torch.Size([125, 358])
torch.Size([125, 361])
torch.Size([125, 357])
torch.Size([125, 448])
torch.Size([125, 360])
torch.Size([125, 362])
torch.Size([125, 360])
torch.Size([125, 357])
torch.Size([125, 357])
torch.Size([125, 359])
torch.Size([125, 443])
torch.Size([125, 439])
torch.Size([125, 452])
torch.Size([125, 449])
torch.Size([125, 452])
torch.Size([125, 454])
torch.Size([125, 450])
torch.Size([125, 464])
torch.Size([125, 485])
torch.Size([125, 493])
torch.Size([125, 451])
torch.Size([125, 480])
torch.Size(

In [288]:
print(len(line_meta_normal_list))
print(len(line_meta_abnormal_list))

243
204


In [293]:
class line_Autoencoder(nn.Module):
    def __init__(self):
        super(line_Autoencoder, self).__init__() # (125, 1000) 이라고 했을 때 
        # Encoder
        self.encoder = nn.Sequential( # Given groups=1, weight of size [16, 128, 3, 3], expected input[1, 32, 8, 95] to have 128 channels, but got 32 channels instead
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 16, 63, 250)
            nn.ReLU(True),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 32, 32, 125)
            nn.ReLU(True),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 64, 16, 63)
            nn.ReLU(True),
            nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 32, 8, 32)
            nn.ReLU(),
            nn.Conv2d(16, 1, kernel_size=3, stride=2, padding=1),  # Output shape: (b, 16, 4, 16)
            
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 16, kernel_size=3, stride=2, padding=1, output_padding=1),  # Output shape: (b, 16, 8, 32)  
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 32, kernel_size=3, stride=2, padding=(1, 1), output_padding=(1,0)),  # Output shape: (b, 32, 16, 63) 
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=(1,1), output_padding=(1,0)),  # Output shape: (b, 64, 32, 125) 
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 32, 63, 250)  
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=(1,1), output_padding=(0,1)),  # Output shape: (b, 16, 125, 500)  
            
        )
# output padding must be smaller than stride
    def forward(self, x):
        mid = self.encoder(x)
        x = self.decoder(mid)
        return x

In [294]:
def line_train(dataset_list):
    dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    customdataset = CustomDataset(dataset_list)
    train_loader = DataLoader(customdataset, batch_size=32, shuffle=True)


    # Initialize the model, loss function, and optimizer
    autoencoder = line_Autoencoder().to(dev)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

    # Train the autoencoder
    num_epochs = 100
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader): # i-th batch
            inputs = data
            inputs = inputs.to(dev)

            optimizer.zero_grad()

            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'epoch : {epoch}, loss: {running_loss}')


    print('Finished Training')

    encoded_list = []
    with torch.no_grad():
        for i in dataset_list:
            encoded_list.append(autoencoder.encoder(i.to(dev)).flatten())

    return encoded_list
           


In [295]:
encoded_line_normal_list = line_train(line_meta_normal_list)
encoded_line_abnormal_list = line_train(line_meta_abnormal_list)

epoch : 0, loss: 18.79534387588501
epoch : 1, loss: 18.005507469177246
epoch : 2, loss: 16.34283137321472
epoch : 3, loss: 8.980451703071594
epoch : 4, loss: 4.571180433034897
epoch : 5, loss: 3.1808572113513947
epoch : 6, loss: 2.4019822478294373
epoch : 7, loss: 1.9643169939517975
epoch : 8, loss: 1.6765075474977493
epoch : 9, loss: 1.491836428642273
epoch : 10, loss: 1.3597840815782547
epoch : 11, loss: 1.255462035536766
epoch : 12, loss: 1.1719489991664886
epoch : 13, loss: 1.1225342750549316
epoch : 14, loss: 1.0924255549907684
epoch : 15, loss: 1.0715667307376862
epoch : 16, loss: 1.0568200051784515
epoch : 17, loss: 1.0416569113731384
epoch : 18, loss: 1.031990334391594
epoch : 19, loss: 1.020093396306038
epoch : 20, loss: 1.008617989718914
epoch : 21, loss: 0.9991780444979668
epoch : 22, loss: 0.99294213950634
epoch : 23, loss: 0.9882945641875267
epoch : 24, loss: 0.9846357852220535
epoch : 25, loss: 0.9796131327748299
epoch : 26, loss: 0.9754904210567474
epoch : 27, loss: 0.97

In [296]:
line_concated = encoded_line_normal_list + encoded_line_abnormal_list 
line_concated = torch.stack(line_concated).cpu().numpy()

# Run k-means clustering
line_cluster_labels = kmeans.fit_predict(line_concated)

line_normal_cnt={0:0, 1:0} # total should be: 243
line_abnormal_cnt={0:0, 1:0} # 204


for i, cl_label in enumerate(line_cluster_labels):
    if i < len(encoded_line_normal_list):
        line_normal_cnt[cl_label]+= 1
    else:
        line_abnormal_cnt[cl_label] += 1
    
print(line_normal_cnt)
print(line_abnormal_cnt)


/opt/miniconda3/envs/recbole_jh/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


{0: 0, 1: 243}
{0: 204, 1: 0}


In [304]:
idx = 43
print(encoded_line_normal_list[idx])
print(encoded_line_abnormal_list[idx])

tensor([-13.3745, -12.9934, -18.4419, -26.6632, -29.4094, -29.3742, -31.2977,
        -32.9443, -36.6441, -34.8051, -36.3590, -36.4045, -38.8022, -40.6189,
        -40.6635, -24.5543, -10.7123,  -6.6756, -13.1272, -11.0439, -14.8549,
        -16.8325, -17.3081, -20.3440, -22.3437, -22.9538, -22.1248, -22.9636,
        -22.4643, -25.7804, -27.2800, -16.7124, -11.2664,  -6.6650,  -7.5051,
        -12.0915, -17.6410, -21.4354, -24.2447, -27.8242, -29.9402, -31.9817,
        -30.9346, -31.9373, -33.5120, -34.3105, -36.6013, -24.6131, -11.9517,
        -11.2176, -10.1818, -13.0367, -20.0484, -22.6991, -23.8574, -25.6085,
        -27.6730, -28.2072, -29.1583, -31.1095, -32.4300, -32.6938, -33.9177,
        -24.2028], device='cuda:0')
tensor([ 2.5997, 12.7299, 16.0664, 26.2039, 36.6281, 31.4814, 36.4267, 39.0514,
        39.2179, 38.6829, 42.0047, 46.4250, 49.2073, 48.0779, 47.7326, 29.6187,
         4.1852,  5.0790,  7.4126,  8.4095, 19.1328, 24.2020, 26.5203, 29.3995,
        31.7641, 32.73